#  Вложенные функции, замыкания и декораторы

## 1. Функции

**Функция** — часть программы, которую можно вызвать из другого места программы.

Все в Python объекты. И даже функции. Это значит, что у функций есть
- атрибуты
- и методы.

От остальных объектов функции отличаются тем, что их можно вызвать*. Объекты, которые можно вызвать, называют `Callable`-объектами.

*\*С точки зрения синтаксиса еще можно вызывать классы*

### Как определить функцию

In [33]:
# Функция определяется так
def function_name(function_argument):
    """текстовое описание функции"""
    return function_argument+1 # тело функции

Как у любого объекта в python, у функции есть:
- идентификатор,
- тип.

In [26]:
# У функции function_name эти параметры выглядят так
id(function_name), type(function_name)

(4510971560, function)

Все атрибуты и методы функции как объекта можно посмотреть:

In [34]:
import inspect
sorted(inspect.getmembers(function_name))

[('__annotations__', {}),
 ('__call__', <method-wrapper '__call__' of function object at 0x10d594598>),
 ('__class__', function),
 ('__closure__', None),
 ('__code__',
  <code object function_name at 0x10cb579c0, file "<ipython-input-33-868ef1d9c6c1>", line 2>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x10d594598>),
 ('__dict__', {}),
 ('__dir__', <function function.__dir__()>),
 ('__doc__', 'текстовое описание функции'),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x10d594598>),
 ('__format__', <function function.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x10d594598>),
 ('__get__', <method-wrapper '__get__' of function object at 0x10d594598>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x10d594598>),
 ('__globals__',
  {'__name__': '__main__',
   '__doc__': 'Automatically created module for IPython interactive environment',
   '__p

### Как вызвать функцию

In [28]:
#  Вызов функции, ожидаем ответ 2
function_name(1)

2

In [30]:
#  Можно явно вызвать метод call, ожидаем ответ 2
function_name.__call__(1)

2

### Как функции устроены

In [31]:
# Байт-код функции function_name
function_name.__code__.co_code

b'|\x00d\x01\x17\x00S\x00'

In [32]:
# Дизассемблированное тело функции function_name
import dis
dis.dis(function_name)

  3           0 LOAD_FAST                0 (function_argument)
              2 LOAD_CONST               1 (1)
              4 BINARY_ADD
              6 RETURN_VALUE


Если заглянуть во внутренности интерпретатора (CPython), то функция описывается следующей струтурой: https://github.com/python/cpython/blob/3.7/Include/funcobject.h

**Почитать**
1. [The Python Language Reference. Data model](https://docs.python.org/3/reference/datamodel.html#objects-values-and-types)
2. [The Python Language Reference. Inspect live objects](https://docs.python.org/3/library/inspect.html)

## 2. Вложенные функции

**Вложенная функция** — функция, которая определена внутри другой функции.

### Область видимости и LEGB

https://www.google.com/url?sa=i&rct=j&q=&esrc=s&source=images&cd=&cad=rja&uact=8&ved=2ahUKEwjumtSs7prgAhWGxIsKHdS2C4QQjRx6BAgBEAU&url=https%3A%2F%2Fsebastianraschka.com%2FArticles%2F2014_python_scope_and_namespaces.html&psig=AOvVaw1Sw3kd_XY-GCz5nsD61FwU&ust=1549121745003362![image.png](attachment:image.png)

## 3. Замыкания

**Замыкание** — вложенная функция, которая использует локальные переменные внешней функции.

## 4. Декораторы

**Декоратор** — функция, которая принимает другую функцию и что-то возвращает.